In [1]:
import os
import numpy as np
import librosa
import scipy
import math
import pickle

In [2]:
DATASET_16BIT_PATH = "D:/Thesis/UrbanSound8K-16bit/audio-classified"
#DATASET_16BIT_PATH = "D:/Thesis/Keras/Attempt5/subset"

In [3]:
def getListOfFiles(dirpath):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirpath)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirpath, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)        
    
    return allFiles

def urban_labels(Y, fpaths):
    """urban sound dataset labels."""
    urban_label = lambda path: int(os.path.split(path)[-1].split('-')[1])
    for p in fpaths:
        Y = np.append(Y, [urban_label(p)])
    return Y

def load_sound_files(file_paths):

    X,sr = librosa.load(file_paths,res_type='kaiser_fast')
    mfccs=np.mean(librosa.feature.mfcc(y=X,sr=sr,n_mfcc=40).T,axis=0)

    return mfccs

In [4]:
filepaths = np.asarray(getListOfFiles(DATASET_16BIT_PATH))
print("No of entries in our Dataset: ", filepaths.shape)

Y = np.ndarray(0)
labels = urban_labels(Y, filepaths)
print("Labels : ",labels.shape)

with open("filepaths.txt", "wb") as fp:   #Pickling
    pickle.dump(filepaths, fp)
    
with open("labels.txt", "wb") as fp:   #Pickling
    pickle.dump(labels, fp)

No of entries in our Dataset:  (9712,)
Labels :  (9712,)


In [5]:
raw = []
for i in range(len(filepaths)):
    #len(filepaths) when ready
    raw.append(load_sound_files(filepaths[i]))
    if i%100 == 0 :
        print("Raw Sound loaded for :", filepaths[i])

np.save("Attempt5/raw",np.asarray(raw))
#del raw

Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\air_conditioner\100852-0-0-0.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\air_conditioner\13230-0-0-22.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\air_conditioner\146714-0-0-41.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\air_conditioner\162103-0-0-14.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\air_conditioner\177726-0-0-15.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\air_conditioner\178686-0-0-43.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\air_conditioner\189982-0-0-20.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\air_conditioner\204240-0-0-23.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\air_conditioner\57320-0-0-5.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\air_cond

Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\siren\24347-8-0-19.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\siren\34952-8-0-6.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\siren\74364-8-1-22.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\siren\94636-8-0-4.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\street_music\115415-9-0-5.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\street_music\14385-9-0-10.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\street_music\155217-9-1-58.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\street_music\171406-9-0-183.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\street_music\186336-9-0-3.wav
Raw Sound loaded for : D:/Thesis/UrbanSound8K-16bit/audio-classified\street_music\26270-9-0-19.wav
Raw Sound loaded for : D:/Thesis/Urba

In [6]:
#del raw
raw = np.load("Attempt5/raw.npy")

In [21]:
from sklearn.model_selection import train_test_split
XTrain,XTest,YTrain,YTest=train_test_split(raw,labels,test_size=0.2)

from keras.utils import to_categorical

YTrain = to_categorical(YTrain, num_classes=11)
YTest = to_categorical(YTest, num_classes=11)

In [8]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, GlobalAveragePooling2D
from keras import optimizers
from keras.utils import np_utils
from sklearn import metrics 


In [9]:
# copied 
try_model = Sequential()

try_model.add(Dense(256, input_shape=(40,)))
try_model.add(Activation('relu'))
try_model.add(Dropout(0.5))

try_model.add(Dense(256))
try_model.add(Activation('relu'))
try_model.add(Dropout(0.5))

try_model.add(Dense(256))
try_model.add(Activation('relu'))
try_model.add(Dropout(0.5))

try_model.add(Dense(11))
try_model.add(Activation('softmax'))

try_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [10]:
try_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
__________

In [11]:
try_model.fit(np.array(XTrain), np.array(YTrain), batch_size=32, epochs=200, validation_data=(np.array(XTest), np.array(YTest)))

Train on 7769 samples, validate on 1943 samples
Epoch 1/200
7769/7769 [==============================] - 3s 361us/step - loss: 12.8170 - acc: 0.1456 - val_loss: 10.5162 - val_acc: 0.2445
Epoch 2/200
7769/7769 [==============================] - 1s 143us/step - loss: 8.7347 - acc: 0.2238 - val_loss: 2.8190 - val_acc: 0.3459
Epoch 3/200
7769/7769 [==============================] - 1s 150us/step - loss: 2.5632 - acc: 0.2809 - val_loss: 1.8394 - val_acc: 0.4472
Epoch 4/200
7769/7769 [==============================] - 1s 149us/step - loss: 1.8667 - acc: 0.3594 - val_loss: 1.6255 - val_acc: 0.5183
Epoch 5/200
7769/7769 [==============================] - 1s 148us/step - loss: 1.6835 - acc: 0.4233 - val_loss: 1.4822 - val_acc: 0.5656
Epoch 6/200
7769/7769 [==============================] - 1s 154us/step - loss: 1.5496 - acc: 0.4714 - val_loss: 1.3237 - val_acc: 0.5872
Epoch 7/200
7769/7769 [==============================] - 1s 155us/step - loss: 1.4625 - acc: 0.5023 - val_loss: 1.2547 - val_acc

7769/7769 [==============================] - 1s 138us/step - loss: 0.6790 - acc: 0.7803 - val_loss: 0.5272 - val_acc: 0.8296
Epoch 61/200
7769/7769 [==============================] - 1s 138us/step - loss: 0.6826 - acc: 0.7811 - val_loss: 0.5216 - val_acc: 0.8399
Epoch 62/200
7769/7769 [==============================] - 1s 142us/step - loss: 0.6728 - acc: 0.7843 - val_loss: 0.5089 - val_acc: 0.8405
Epoch 63/200
7769/7769 [==============================] - 1s 147us/step - loss: 0.6696 - acc: 0.7808 - val_loss: 0.5147 - val_acc: 0.8405
Epoch 64/200
7769/7769 [==============================] - 1s 145us/step - loss: 0.6651 - acc: 0.7831 - val_loss: 0.5227 - val_acc: 0.8353
Epoch 65/200
7769/7769 [==============================] - 2s 200us/step - loss: 0.6686 - acc: 0.7845 - val_loss: 0.4870 - val_acc: 0.8533
Epoch 66/200
7769/7769 [==============================] - 1s 144us/step - loss: 0.6814 - acc: 0.7840 - val_loss: 0.4929 - val_acc: 0.8420
Epoch 67/200
7769/7769 [=======================

7769/7769 [==============================] - 1s 136us/step - loss: 0.5711 - acc: 0.8195 - val_loss: 0.4241 - val_acc: 0.8564
Epoch 120/200
7769/7769 [==============================] - 1s 153us/step - loss: 0.5829 - acc: 0.8090 - val_loss: 0.4308 - val_acc: 0.8595
Epoch 121/200
7769/7769 [==============================] - 1s 138us/step - loss: 0.5661 - acc: 0.8161 - val_loss: 0.4351 - val_acc: 0.8580
Epoch 122/200
7769/7769 [==============================] - 1s 139us/step - loss: 0.5844 - acc: 0.8170 - val_loss: 0.4366 - val_acc: 0.8672
Epoch 123/200
7769/7769 [==============================] - 1s 136us/step - loss: 0.5880 - acc: 0.8159 - val_loss: 0.4516 - val_acc: 0.8538
Epoch 124/200
7769/7769 [==============================] - 1s 136us/step - loss: 0.5969 - acc: 0.8126 - val_loss: 0.4214 - val_acc: 0.8724
Epoch 125/200
7769/7769 [==============================] - 1s 135us/step - loss: 0.6121 - acc: 0.8069 - val_loss: 0.4561 - val_acc: 0.8549
Epoch 126/200
7769/7769 [================

7769/7769 [==============================] - 1s 152us/step - loss: 0.5573 - acc: 0.8276 - val_loss: 0.4195 - val_acc: 0.8652
Epoch 179/200
7769/7769 [==============================] - 1s 148us/step - loss: 0.5506 - acc: 0.8337 - val_loss: 0.4175 - val_acc: 0.8585
Epoch 180/200
7769/7769 [==============================] - 1s 145us/step - loss: 0.5616 - acc: 0.8322 - val_loss: 0.4210 - val_acc: 0.8662
Epoch 181/200
7769/7769 [==============================] - 1s 149us/step - loss: 0.5662 - acc: 0.8248 - val_loss: 0.4135 - val_acc: 0.8672
Epoch 182/200
7769/7769 [==============================] - 1s 146us/step - loss: 0.5493 - acc: 0.8264 - val_loss: 0.4060 - val_acc: 0.8693
Epoch 183/200
7769/7769 [==============================] - 1s 146us/step - loss: 0.5287 - acc: 0.8345 - val_loss: 0.4172 - val_acc: 0.8657
Epoch 184/200
7769/7769 [==============================] - 1s 151us/step - loss: 0.5270 - acc: 0.8310 - val_loss: 0.3975 - val_acc: 0.8708
Epoch 185/200
7769/7769 [================

In [27]:
#On training Set
train_evaluate = try_model.evaluate(x=np.array(XTrain), y=np.array(YTrain), batch_size=32)
print(train_evaluate)
#On test Set
test_evaluate = try_model.evaluate(x=np.array(XTest), y=np.array(YTest), batch_size=32)
print(test_evaluate)

7769/7769 [==============================] - 0s 50us/step
[0.3007816478681006, 0.9097695971704508]
1943/1943 [==============================] - 0s 51us/step
[0.3046217371416779, 0.9068450850429329]


In [26]:
#voila

In [30]:
#from sklearn.metrics import classification_report

#YPred = try_model.predict(XTest)
#print(classification_report(YTest, YPred))